In [ ]:
from dask.base import visualize, compute

In [ ]:
from dask.distributed import Client, progress
client = Client()
client

In [ ]:
client.scheduler_info()
# client.write_scheduler_file('scheduler.json')

In [ ]:
client.cluster

## The task function

In [ ]:
import dask
from typing import Dict, Any
from copy import deepcopy
import time


def sleeper(input_1=None, input_2=2, input_3=False):
    #
    # run container and pass parameters
    #
    print(input_2+1)
    time.sleep(input_2+1)
    outputs = {"output_1": input_1, "output_2": input_2}
    return outputs

### single task run

In [ ]:
future = client.submit(sleeper, input_1=33, input_2=1)
progress(future)

In [ ]:
future.result()

### pipeline run

This is how it would look assembling it by hand

In [ ]:
from dask.base import visualize

outs = []
out98 = dask.delayed(sleeper)(input_2=10)
oute9 = dask.delayed(sleeper)(input_2=10)

outc7 = dask.delayed(sleeper)(input_1=out98["output_1"], input_2=out98["output_2"])
out53 = dask.delayed(sleeper)(input_1=out98["output_1"], input_2=out98["output_2"])
out40 = dask.delayed(sleeper)(input_1=out98["output_1"], input_2=out98["output_2"])
out2d = dask.delayed(sleeper)(input_1=out98["output_1"], input_2=out98["output_2"])

outaf = dask.delayed(sleeper)(input_1=oute9["output_1"], input_2=oute9["output_2"])


outs = [out98, oute9, outc7, out53, out40, out2d, outaf]
visualize(outs, rankdir='LR', verbose=True)

In [ ]:
f = dask.persist(outs)
progress(f)

In [ ]:
client.who_has(f)

In [ ]:
import json

with open("project.json") as fp:
    project = json.load(fp)

workbench = project["workbench"]


In [ ]:
nouts = {}
def get_val(v):
    if isinstance(v, dict):
        return nouts[v["nodeUuid"]][v["output"]]
    return v


for nid, node in workbench.items():
    nins = {name: get_val(value) for name, value in node["inputs"].items()}
    nouts[nid] = dask.delayed(sleeper)(**nins)



In [ ]:
visualize( list(nouts.values()), rankdir='LR', collapse_outputs=False, optimize_graph=True, verbose=True)

In [ ]:
p = dask.persist(list(nouts.values()))

In [ ]:
progress(p)

In [ ]:
del nouts
del p


In [ ]:
def get_number_of_tasks(dask_scheduler=None):
    return len(dask_scheduler.tasks)

client.run_on_scheduler(get_number_of_tasks)